In [1]:
# Импорт необходимых библиотек

import pandas as pd
import sqlite3
import requests

In [2]:
# Создайте базу данных sqlite, можно использовать sqlalchemy или pandas, или любой другой инструмент

CONN = sqlite3.connect('kursach_0.db')

In [8]:
# Список всех таблиц БД

all_table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", CONN)
all_table

,name


In [7]:
# Удаление таблиц, функция ниже

sql_query(sql='DROP TABLE IF EXISTS JOIN_DATA', conn=CONN)

In [9]:
# Напишите функцию которая принимает на вход дату
# и выгружает курс валюты EUR к USD за этот день
# Функция должна вернуть список словарей или можно использовать можно использоват pandas.read_csv()

# Для этого воспользуйтесь https://exchangerate.host/#/
# и методом Timeseries Endpoint! Строго им. Где start_date = end_date = date
# Используйте параметры start_date, end_date, base=EUR, symbols=USD, format=csv

def extract_currency(date):
    url = 'https://api.exchangerate.host/timeseries?start_date=' + date + '&end_date=' + date + '&base=EUR&symbols=USD'
    response = requests.get(url)
    data = response.json()
    
    df_exchangerate = pd.DataFrame(columns = ['base', 'start_date', 'end_date','rates_usd'])
    df_exchangerate = df_exchangerate.append({'base':data['base'], 'start_date':data['start_date'], 'end_date':data['end_date'],
                                             'rates_usd':data['rates'][date]['USD']}, ignore_index = True)
    
    return df_exchangerate

In [10]:
# Напишите функцию которая скачивает данные с гитхаба
# Функция принимает на вход дату
# Функция должна вернуть список словарей или можно использовать можно использоват pandas.read_csv()

# ссылка на гитхаб с файлами - https://github.com/dm-novikov/stepik_airflow_course/tree/main/data_new


# date строго один из списка 2021-01-01, 2021-01-02, 2021-01-03, 2021-01-04 
def extract_data(date):
    
    url = 'https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/' + date + '.csv'
    
    df_git = pd.DataFrame(columns = ['currency', 'value', 'date'])
    df_git = pd.read_csv(url, on_bad_lines='skip')
    return df_git
    

In [11]:
# Напишите функцию которая принимает на вход список словарей
# и загружает в табличку в sqlite, можно использоват pandas.to_sql()

def insert_to_db(data, table_name, conn):
    #cur = conn.cursor()
    data.to_sql(name=table_name, con=conn, index=False, if_exists='append')

In [12]:
# Напишите функцию которая выполнит SQL запрос к базу данных
# Здесь удобно использовать sqlalchemy или https://pythonru.com/biblioteki/vvedenie-v-sqlite-python
# Если выполняется select ... то функция должна возвращать результат (не None)
# Если используется внутренняя операция например truncate table, insert into select ... то возвращает None
# data = sql_query(sql="select * from table") -> [(1, 2, 3), (4, 5, 6) ...]
# sql_query("truncate table") -> None

#query_sql_1 = 'select * from table_git'
def sql_query(sql, conn): 
    query_split = sql.split(' ')
    
    if 'select' == query_split[0] or 'SELECT' == query_split[0] or 'Select' == query_split[0]:
        data = pd.read_sql(sql, conn)
        return data
    
    elif 'insert'== query_split[0] or 'INSERT' == query_split[0] or 'Insert'== query_split[0]:
        
        name_new_tb_JOIN_DATA = query_split[2]
        
        query_select_split = query_split
        del query_select_split[0:3]
        query_select_str = ' '.join(query_select_split)
        
        data = pd.read_sql(query_select_str, conn)
        
        data.to_sql(name=name_new_tb_JOIN_DATA, con=conn, index=False, if_exists='append')       
        return None
    
    elif 'Drop'== query_split[0] or 'drop' == query_split[0] or 'DROP'== query_split[0]:
        #print('drop')
        cur = conn.cursor()
        cur.execute(sql)
        #conn.close()        # не стоит закрывать доступ к бд пока с ней работают
        return(None)
    
    elif 'Truncate'== query_split[0] or 'truncate' == query_split[0] or 'TRUNCATE'== query_split[0]:
        print('Ничего не произошло с БД, в sqlite нет Truncate - используйте конструкцию DROP TABLE IF EXISTS table_name')
        return(None)
    else:
        print('Ничего не произошло с БД, смотри функцию sql_query - она поддерживает запросы: select, insert, drop')
        return(None)


In [13]:
# Запустите ваш код в функции main

# Напишите генерацию дат, так чтобы у вас получился список
# 2021-01-02, 2021-01-03 ... etc
# Нужны даты с 2021-01-01 по 2021-01-04

dates_list = list(['2021-01-01','2021-01-02','2021-01-03','2021-01-04'])

def main(date, conn):
  
  # Выгружаем данные по валютам и из источника
  currency = extract_currency(date)
  data = extract_data(date)
  # Создайте необходимые таблицы если это нужно
  # 'currency', 'data' и 'join_data (таблица с объединенными данными)
  #sql_query(sql='select * from table_git', conn) 


  # Вставляем данные в БД
  # У вас долнжо получиться 2 таблицы CURRENCY, DATA
  insert_to_db(data=currency, table_name='currency', conn=conn)
  
  insert_to_db(data=data, table_name='data', conn=conn)
 

  all_table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", CONN)
  

  # Объединение данных в отдельную таблицу JOIN_DATA
  # CURRENCY, DATA объединить через JOIN по дате и валюте
  # Используйте insert into select ... inner join ...
  # И соответственно создать таблицу для этого
  sql_query(sql='INSERT INTO JOIN_DATA SELECT * FROM currency INNER JOIN data ON date=start_date', conn=CONN)
  
  all_table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", CONN)
  
  # Очистка временных таблиц CURRENCY, DATA
  # Это те таблицы куда произошла вставка данных
  # Используйте truncate
  sql_query(sql='DROP TABLE IF EXISTS currency', conn=CONN)
  sql_query(sql='DROP TABLE IF EXISTS data', conn=CONN)
  # Получение данных из таблицы JOIN_DATA за конкретный день
  # Используйте date при формировании запроса select * from ... where date ='...'
  report = sql_query(sql="select * from JOIN_DATA where date = '2021-01-01'", conn=CONN)
  print(report)

# Пройдемся по списку дат и выполним скрипт
for date in dates_list:
  main(date, conn=CONN)

C:\Users\Ejik\AppData\Local\Temp\ipykernel_3120\584436523.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_exchangerate = df_exchangerate.append({'base':data['base'], 'start_date':data['start_date'], 'end_date':data['end_date'],


  base  start_date    end_date  rates_usd currency  value        date
0  EUR  2021-01-01  2021-01-01   1.217582      EUR     23  2021-01-01
1  EUR  2021-01-01  2021-01-01   1.217582      EUR     24  2021-01-01
2  EUR  2021-01-01  2021-01-01   1.217582      EUR     38  2021-01-01
3  EUR  2021-01-01  2021-01-01   1.217582      EUR     42  2021-01-01
4  EUR  2021-01-01  2021-01-01   1.217582      EUR     48  2021-01-01
5  EUR  2021-01-01  2021-01-01   1.217582      EUR     65  2021-01-01
6  EUR  2021-01-01  2021-01-01   1.217582      EUR     74  2021-01-01
7  EUR  2021-01-01  2021-01-01   1.217582      EUR     74  2021-01-01
8  EUR  2021-01-01  2021-01-01   1.217582      EUR     86  2021-01-01


C:\Users\Ejik\AppData\Local\Temp\ipykernel_3120\584436523.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_exchangerate = df_exchangerate.append({'base':data['base'], 'start_date':data['start_date'], 'end_date':data['end_date'],


  base  start_date    end_date  rates_usd currency  value        date
0  EUR  2021-01-01  2021-01-01   1.217582      EUR     23  2021-01-01
1  EUR  2021-01-01  2021-01-01   1.217582      EUR     24  2021-01-01
2  EUR  2021-01-01  2021-01-01   1.217582      EUR     38  2021-01-01
3  EUR  2021-01-01  2021-01-01   1.217582      EUR     42  2021-01-01
4  EUR  2021-01-01  2021-01-01   1.217582      EUR     48  2021-01-01
5  EUR  2021-01-01  2021-01-01   1.217582      EUR     65  2021-01-01
6  EUR  2021-01-01  2021-01-01   1.217582      EUR     74  2021-01-01
7  EUR  2021-01-01  2021-01-01   1.217582      EUR     74  2021-01-01
8  EUR  2021-01-01  2021-01-01   1.217582      EUR     86  2021-01-01


C:\Users\Ejik\AppData\Local\Temp\ipykernel_3120\584436523.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_exchangerate = df_exchangerate.append({'base':data['base'], 'start_date':data['start_date'], 'end_date':data['end_date'],


  base  start_date    end_date  rates_usd currency  value        date
0  EUR  2021-01-01  2021-01-01   1.217582      EUR     23  2021-01-01
1  EUR  2021-01-01  2021-01-01   1.217582      EUR     24  2021-01-01
2  EUR  2021-01-01  2021-01-01   1.217582      EUR     38  2021-01-01
3  EUR  2021-01-01  2021-01-01   1.217582      EUR     42  2021-01-01
4  EUR  2021-01-01  2021-01-01   1.217582      EUR     48  2021-01-01
5  EUR  2021-01-01  2021-01-01   1.217582      EUR     65  2021-01-01
6  EUR  2021-01-01  2021-01-01   1.217582      EUR     74  2021-01-01
7  EUR  2021-01-01  2021-01-01   1.217582      EUR     74  2021-01-01
8  EUR  2021-01-01  2021-01-01   1.217582      EUR     86  2021-01-01


C:\Users\Ejik\AppData\Local\Temp\ipykernel_3120\584436523.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_exchangerate = df_exchangerate.append({'base':data['base'], 'start_date':data['start_date'], 'end_date':data['end_date'],


  base  start_date    end_date  rates_usd currency  value        date
0  EUR  2021-01-01  2021-01-01   1.217582      EUR     23  2021-01-01
1  EUR  2021-01-01  2021-01-01   1.217582      EUR     24  2021-01-01
2  EUR  2021-01-01  2021-01-01   1.217582      EUR     38  2021-01-01
3  EUR  2021-01-01  2021-01-01   1.217582      EUR     42  2021-01-01
4  EUR  2021-01-01  2021-01-01   1.217582      EUR     48  2021-01-01
5  EUR  2021-01-01  2021-01-01   1.217582      EUR     65  2021-01-01
6  EUR  2021-01-01  2021-01-01   1.217582      EUR     74  2021-01-01
7  EUR  2021-01-01  2021-01-01   1.217582      EUR     74  2021-01-01
8  EUR  2021-01-01  2021-01-01   1.217582      EUR     86  2021-01-01


In [ ]:
# чтобы првоерить решение можете обратиться к вашей базе данных таким образом
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql sqlite:////<ПУТЬ ДО БАЗЫ>
%sql select * from <ТАБЛИЦА>